# RNN 3D - x-collapsed
# adapt from RNN 4D


In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, AdditiveAttention, LayerNormalization
from functools import partial
from tensorflow.keras import models, layers

c:\Users\josep\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [5]:
# import nibabel as nib
# import os
# import shutil

# # Directory containing your .nii.gz files
# directory_path = '4D/'
# file_pattern = '*.nii.gz'  # Adjust as needed for your file pattern

# # Directory to move corrupt files, create if doesn't exist
# corrupt_files_dir = os.path.join(directory_path, 'corrupt_files')
# os.makedirs(corrupt_files_dir, exist_ok=True)

# # List to store paths of corrupt files
# corrupt_files = []

# # Iterate over all files in the directory
# for root, _, files in os.walk(directory_path):
#     for file in files:
#         if file.endswith('.nii.gz'):
#             file_path = os.path.join(root, file)
#             try:
#                 # Attempt to load the file
#                 t1_img = nib.load(file_path)
#                 # Attempt to read the data to ensure it's not corrupt
#                 t1_data = t1_img.get_fdata()

#             except (EOFError, OSError, nib.filebasedimages.ImageFileError) as e:
#                 # Log the corrupt file and the error message
#                 print(f"Corrupt file detected: {file_path} | Error: {e}")
#                 corrupt_files.append(file_path)

#                 # Optionally, move the corrupt file to a separate directory
#                 shutil.move(file_path, os.path.join(corrupt_files_dir, file))
#                 continue

# # Output the list of corrupt files
# if corrupt_files:
#     print(f"\nTotal corrupt files found: {len(corrupt_files)}")
#     for corrupt_file in corrupt_files:
#         print(corrupt_file)
# else:
#     print("No corrupt files found.")


In [3]:
import os
import glob
import random
import numpy as np
import nibabel as nib

# GAN Training Data Selection
gan_train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 5)
gan_test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in gan_train_ids_schiz]

gan_train_ids_control = random.sample(met_requirement_control_ids, 5)
gan_test_ids_control = [id for id in met_requirement_control_ids if id not in gan_train_ids_control]
gan_test_ids_control = random.sample(gan_test_ids_control, 5)

# Classifier Test Data Selection
classifier_test_ids = gan_test_ids_schiz + gan_test_ids_control

# Specify the directory and file pattern
directory_path = '4D/'
file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

# Construct the full path pattern
path_pattern = f'{directory_path}/{file_pattern}'

# Use glob to find all matching files
matching_files = glob.glob(path_pattern)

# Lists to store data for GAN training and classification
gan_image_data_schiz = []
gan_image_data_control = []

# Iterate through matching files and load data
for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in gan_train_ids_schiz or file_id in gan_train_ids_control:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()
        
        # Filter out data with insufficient time points
        if t1_data.shape[3] < 90:
            continue

        # Collapse the x-axis by summing over it
        t1_data_collapsed = np.sum(t1_data, axis=0)  # Sum over x-axis (axis=0)

        # Store collapsed data in the appropriate list
        if file_id in gan_train_ids_schiz:
            gan_image_data_schiz.append(t1_data_collapsed)
        elif file_id in gan_train_ids_control:
            gan_image_data_control.append(t1_data_collapsed)

# Display total loaded data counts
print(f"Total GAN control loaded: {len(gan_image_data_control)}")
print(f"Total GAN schiz loaded: {len(gan_image_data_schiz)}")

# Determine the maximum time-dimension size
max_t_size_schiz = max(img.shape[2] for img in gan_image_data_schiz)  # Time dimension is now at index 2 after collapsing
max_t_size_control = max(img.shape[2] for img in gan_image_data_control)
max_t_size = max(max_t_size_schiz, max_t_size_control)

# Normalize and pad the data
def normalize_and_pad(data, max_t):
    """
    Normalize and pad the MRI data to ensure consistent shape.
    
    Args:
    - data (numpy array): Input MRI data.
    - max_t (int): Maximum time dimension size to pad to.
    
    Returns:
    - padded (numpy array): Normalized and padded MRI data.
    """
    # Normalize data to range [-1, 1]
    normalized = (data - np.min(data)) / (np.max(data) - np.min(data)) * 2 - 1
    # Pad the time dimension to max_t
    padded = np.pad(normalized, ((0, 0), (0, 0), (0, max_t - data.shape[2])), mode='constant')
    return padded

# Apply normalization and padding to all loaded data
padded_data_schiz = [normalize_and_pad(img, max_t_size) for img in gan_image_data_schiz]
padded_data_control = [normalize_and_pad(img, max_t_size) for img in gan_image_data_control]

# Convert lists to numpy arrays for model input
padded_data_array_schiz = np.array(padded_data_schiz)
padded_data_array_control = np.array(padded_data_control)

# Print the shape of the padded arrays
print(f"Shape after normalization and padding (control): {padded_data_array_control[0].shape}")
print(f"Shape after normalization and padding (schiz): {padded_data_array_schiz[0].shape}")


Total GAN control loaded: 5
Total GAN schiz loaded: 5
Shape after normalization and padding (control): (109, 91, 146)
Shape after normalization and padding (schiz): (109, 91, 146)


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Enabled memory growth for GPUs.")
    except RuntimeError as e:
        print(e)

Enabled memory growth for GPUs.


In [5]:
import numpy as np
import tensorflow as tf

# Batch size
batch_size = 4

# Create labels
labels_schiz = np.ones(len(padded_data_array_schiz))
labels_control = np.zeros(len(padded_data_array_control))

# Combine images and labels
train_images = np.concatenate((padded_data_array_schiz, padded_data_array_control), axis=0)  # Concatenate along the first dimension
train_labels = np.concatenate((labels_schiz, labels_control), axis=0)

# Shuffle indices
indices = np.arange(len(train_images))
np.random.shuffle(indices)

# Shuffle data based on indices
train_images = [train_images[i] for i in indices]
train_labels = train_labels[indices]

# Define a generator function to yield data batches
def data_generator(images, labels, batch_size):
    """
    Generator function to yield batches of images and labels.

    Args:
    - images (list of numpy arrays): List of MRI images.
    - labels (numpy array): Array of labels corresponding to the images.
    - batch_size (int): Number of samples per batch.

    Yields:
    - batch_images (numpy array): Batch of MRI images.
    - batch_labels (numpy array): Batch of labels.
    """
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]
        
        # Convert list of arrays to numpy array and add channel dimension
        # New shape should be (batch_size, time_steps, y, z, 1)
        batch_images = np.array(batch_images).transpose(0, 3, 1, 2)[..., np.newaxis]  # Adjust axes after x-axis collapse
        yield batch_images, np.array(batch_labels)

# Create TensorFlow Dataset from the generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_images, train_labels, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, max_t_size, 109, 91, 1), dtype=tf.float32),  # Adjusted shape for ConvLSTM3D after collapsing x-axis
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

# Prefetch for performance improvement
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Debug: Test the generator
for images, labels in train_dataset.take(1):
    print(f"Batch image shape: {images.shape}")  # Should be (batch_size, max_t_size, 109, 91, 1)
    print(f"Batch labels shape: {labels.shape}")


Batch image shape: (4, 146, 109, 91, 1)
Batch labels shape: (4,)


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Available GPUs: {[gpu.name for gpu in gpus]}")
else:
    print("No GPUs detected.")

# Define the RNN model using ConvLSTM2D
def build_rnn_model():
    # Define input shape: (time_steps, y, z, channels)
    time_steps = max_t_size  # Number of time points (calculated previously)
    input_shape = (time_steps, 109, 91, 1)  # Shape after collapsing x-axis and adding channel

    # Input layer
    inputs = layers.Input(shape=input_shape)
    
    # ConvLSTM2D layer to process 2D spatial-temporal data
    convlstm_out = layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding='same', return_sequences=True, data_format='channels_last')(inputs)

    # Use TimeDistributed to apply 2D pooling to each time step
    pooled_out = tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAveragePooling2D())(convlstm_out)
    
    # Dense layer with reduced size
    dense_out = tf.keras.layers.Dense(8, activation='relu')(pooled_out)
    
    # Output layer for binary classification
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense_out)

    # Compile model
    model = tf.keras.models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-7),
                  loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Create the model
rnn_model = build_rnn_model()
rnn_model.summary()

# Metrics for training
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

# Define number of epochs
epochs = 2

# Define a step function for training
@tf.function
def train_step(images, labels):
    """
    Perform a single training step.

    Args:
    - images (tf.Tensor): Batch of images.
    - labels (tf.Tensor): Corresponding labels.

    Returns:
    - None
    """
    with tf.GradientTape() as tape:
        predictions = rnn_model(images, training=True)
        # Compute binary cross-entropy loss
        loss = tf.keras.losses.binary_crossentropy(labels, predictions[:, -1, 0])

    # Apply gradients
    gradients = tape.gradient(loss, rnn_model.trainable_variables)
    rnn_model.optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))

    # Update training metrics
    train_loss.update_state(loss)
    train_accuracy.update_state(labels, predictions[:, -1, 0])

# Training loop
for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()

    # Iterate through the training dataset
    for images, labels in train_dataset:
        train_step(images, labels)

    # Print the loss and accuracy for the current epoch
    print(f"Epoch {epoch + 1}, Loss: {train_loss.result().numpy()}, Training Accuracy: {train_accuracy.result().numpy()}")


Available GPUs: ['/physical_device:GPU:0']
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 146, 109, 91, 1)  0         
                             ]                                   
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 146, 109, 91, 16)  9856      
                                                                 
 time_distributed (TimeDistr  (None, 146, 16)          0         
 ibuted)                                                         
                                                                 
 dense (Dense)               (None, 146, 8)            136       
                                                                 
 dense_1 (Dense)             (None, 146, 1)            9         
                                                                 
Total params: 10,0

ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv_lstm2d/while/clip_by_value_1' defined at (most recent call last):
    File "c:\Users\josep\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\josep\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\josep\anaconda3\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\josep\anaconda3\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\josep\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\josep\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\josep\anaconda3\lib\asyncio\base_events.py", line 603, in run_forever
     

In [7]:
import numpy as np
import nibabel as nib
import tensorflow as tf
import os

# List to store test images and labels
test_image_data = []
test_labels = []

# Test IDs to filter relevant files
test_ids = classifier_test_ids  # Assuming classifier_test_ids is defined earlier in the script

# Iterate through matching files and filter based on test IDs
for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in test_ids:
        # Load the MRI image
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()
        
        # Filter out images with insufficient time points
        if t1_data.shape[3] < 90:
            continue

        # Collapse the x-axis by summing over it
        t1_data_collapsed = np.sum(t1_data, axis=0)  # Sum over x-axis (axis=0)
        
        # Normalize the processed image (shape: (109, 91, t))
        processed_image_normalized = (t1_data_collapsed - np.min(t1_data_collapsed)) / (np.max(t1_data_collapsed) - np.min(t1_data_collapsed)) * 2 - 1

        # Pad or truncate the time dimension to match the expected size (max_t_size)
        current_t_size = processed_image_normalized.shape[2]
        if current_t_size < max_t_size:
            pad_size = max_t_size - current_t_size
            processed_image_padded = np.pad(
                processed_image_normalized, 
                ((0, 0), (0, 0), (0, pad_size)), 
                mode='constant'
            )
        elif current_t_size > max_t_size:
            processed_image_padded = processed_image_normalized[:, :, :max_t_size]
        else:
            processed_image_padded = processed_image_normalized

        # Reshape to add channel dimension (shape: (109, 91, max_t_size, 1))
        processed_image_padded = np.expand_dims(processed_image_padded, axis=-1)  # Add channel dimension

        # Transpose to match (time, y, z, channels) for ConvLSTM2D (shape: (max_t_size, 109, 91, 1))
        processed_image_transposed = np.transpose(processed_image_padded, (2, 0, 1, 3))

        # Append to test image data
        test_image_data.append(processed_image_transposed)
        
        # Determine the label: 1 for schizophrenia, 0 for control
        label = 1 if file_id in met_requirement_schizophrenia_ids else 0
        test_labels.append(label)

# Convert lists to numpy arrays for easier handling in TensorFlow
test_images_array = np.array(test_image_data)
test_labels_array = np.array(test_labels)

# Create a TensorFlow dataset from the numpy arrays
batch_size = 1
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)

# Prefetch for performance improvement
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Debug: Check the shapes of the test data
for images, labels in test_dataset.take(1):
    print(f"Batch image shape: {images.shape}")  # Expected shape: (batch_size, max_t_size, 109, 91, 1)
    print(f"Batch labels shape: {labels.shape}")  # Expected shape: (batch_size,)

Batch image shape: (1, 146, 109, 91, 1)
Batch labels shape: (1,)


In [8]:
# Evaluate the model
# Define evaluation step without using tf.function
def evaluation_step(images, labels):
    """
    Perform a single evaluation step.

    Args:
    - images (tf.Tensor): Batch of images.
    - labels (tf.Tensor): Corresponding labels.

    Returns:
    - None
    """
    # Get predictions from the model
    predictions = rnn_model(images, training=False)
    
    # Compute binary cross-entropy loss
    loss = tf.keras.losses.binary_crossentropy(labels, predictions[:, -1, 0])
    
    # Update evaluation metrics
    test_loss.update_state(loss)
    test_accuracy.update_state(labels, predictions[:, -1, 0])

# Initialize metrics for evaluation
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

# Reset states before evaluating
test_loss.reset_states()
test_accuracy.reset_states()

# Evaluate on test_dataset without using strategy.run()
for images, labels in test_dataset:
    evaluation_step(images, labels)

# Print evaluation results
print(f"Test Loss: {test_loss.result().numpy()}, Test Accuracy: {test_accuracy.result().numpy()}")

Test Loss: 0.69437175989151, Test Accuracy: 0.3050847351551056


In [12]:
# Predict the probabilities on the test dataset
predictions = rnn_model.predict(test_dataset)

# Print shape of predictions for debugging
print(f"Predictions shape: {predictions.shape}")

# Convert probabilities to class labels (binary classification)
predicted_labels = (predictions > 0.5).astype(int)

# Extract the last time step prediction for each sequence
# Assuming predictions shape is (num_sequences, max_t_size, 1)
predicted_labels_last = predicted_labels[:, -1, 0]  # Extract last time step and flatten

# Print the shape of predicted_labels_last for debugging
print(f"Predicted labels shape (last time step): {predicted_labels_last.shape}")

# Extract actual labels for comparison (actual labels are already at the sequence level)
actual_labels_per_sequence = actual_labels.flatten()  # Just flatten, no slicing needed

# Print the shape of extracted actual labels for comparison
print(f"Actual labels per sequence shape: {actual_labels_per_sequence.shape}")

# Check if the lengths of actual and predicted labels match before evaluation
if len(actual_labels_per_sequence) == len(predicted_labels_last):
    from sklearn.metrics import classification_report, confusion_matrix

    # Compute and print confusion matrix
    print("Confusion Matrix:")
    print(confusion_matrix(actual_labels_per_sequence, predicted_labels_last))
    
    # Compute and print classification report
    print("Classification Report:")
    print(classification_report(actual_labels_per_sequence, predicted_labels_last))
else:
    print("The lengths of actual labels per sequence and predicted labels still don't match.")
    print(f"Length of actual labels per sequence: {len(actual_labels_per_sequence)}")
    print(f"Length of predicted labels: {len(predicted_labels_last)}")


59/59 [==============================] - 5s 82ms/step
Predictions shape: (59, 146, 1)
Predicted labels shape (last time step): (59,)
Actual labels per sequence shape: (59,)
Confusion Matrix:
[[ 5  0]
 [41 13]]
Classification Report:
              precision    recall  f1-score   support

           0       0.11      1.00      0.20         5
           1       1.00      0.24      0.39        54

    accuracy                           0.31        59
   macro avg       0.55      0.62      0.29        59
weighted avg       0.92      0.31      0.37        59



In [13]:
import pandas as pd

# Create a DataFrame to compare predicted vs. actual labels
comparison_df = pd.DataFrame({
    'Predicted Labels': predicted_labels_last,
    'Actual Labels': actual_labels_per_sequence
})

# Display the first few rows of the comparison
print(comparison_df.head())

# Print the full comparison DataFrame
print(comparison_df)

   Predicted Labels  Actual Labels
0                 0              0
1                 0              1
2                 0              1
3                 0              1
4                 0              1
    Predicted Labels  Actual Labels
0                  0              0
1                  0              1
2                  0              1
3                  0              1
4                  0              1
5                  1              1
6                  0              0
7                  0              1
8                  0              1
9                  0              0
10                 0              1
11                 0              1
12                 1              1
13                 0              1
14                 1              1
15                 1              1
16                 0              1
17                 0              1
18                 0              1
19                 1              1
20                 0              